In [1]:
import pandas as pd
from underthesea import word_tokenize
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
from numpy import hstack
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import hstack

In [2]:
df = pd.read_csv("Dataset_articles_NoID.csv")
df

,URL,Title,Summary,Contents,Date,Author(s),Category,Tags
0,https://laodong.vn/bat-dong-san/thong-tin-ngoc...,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",Phương Nhiên,Bất động sản,"['Lâm Đồng', 'Ngọc Trinh', 'Chiêu trò', 'Giá đ..."
1,https://laodong.vn/bat-dong-san/lo-hong-trong-...,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",Gia Miêu,Bất động sản,"['Thủ Thiêm', 'Đấu giá đất']"
2,https://laodong.vn/bat-dong-san/som-hoan-thien...,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,"['Dự án', 'Nhà ở xã hội', 'Dự án nhà ở xã hội'..."
3,https://laodong.vn/bat-dong-san/chi-tiet-ho-so...,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",Kim Nhung (T/H),Bất động sản,"['Giấy phép xây dựng', 'Hồ sơ hoàn công', 'nhà..."
4,https://laodong.vn/bat-dong-san/khoi-tao-khong...,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",Huyền Nguyễn,Bất động sản,['An Quý Villa']
...,...,...,...,...,...,...,...,...
313315,https://laodong.vn/tlv-canh-doi/ba-lao-mu-nuoi...,"Bà lão mù nuôi con, cháu bị ảnh hưởng chất độc...","Bà Dương Thị Tuyết ở thị trấn Khe Sanh, huyện ...",NaN,"Thứ tư, 14/08/2013 06:57 (GMT+7)",Lâm Hưng Thơ,Tấm Lòng Vàng,['Chất độc']
313316,https://laodong.vn/ho-tro-bao-ve-chu-quyen-bie...,Trao 100 triệu đồng cho ngư dân Phú Yên,"Ngày 25.7, Đại diện Chương trình “Tấm lưới ngh...",NaN,"Thứ sáu, 26/07/2013 09:03 (GMT+7)",Lưu Phong,Tin hoạt động,['Phú Yên']
313317,https://laodong.vn/giup-do-cac-canh-doi-kho-kh...,Trao 2 “Mái ấm công đoàn” tới CNVCLĐ khó khăn ...,"Ngày 17.7, lãnh đạo Quỹ TLV Lao Động và LĐLĐ t...",NaN,"Thứ năm, 18/07/2013 07:37 (GMT+7)",Bảo Duy,Tin hoạt động,"['Mái ấm Công đoàn', 'Bắc Giang']"
313318,https://laodong.vn/tlv-canh-doi/hay-giup-do-mo...,Hãy giúp đỡ một nhà báo cần thay thận,Nhà báo Trần Đình Chính (tức Trần Hoài Thu) - ...,NaN,"Thứ năm, 20/06/2013 07:28 (GMT+7)",Hoàng Mạnh - Phạm Tùng,Tấm Lòng Vàng,"['Nhà báo', 'Báo']"


In [3]:
#Bỏ cột URL
df = df.drop("URL", axis=1)

In [4]:
#Loại bỏ hàng NULL/NA
df = df.dropna(axis=0, how='any')
df

,Title,Summary,Contents,Date,Author(s),Category,Tags
0,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",Phương Nhiên,Bất động sản,"['Lâm Đồng', 'Ngọc Trinh', 'Chiêu trò', 'Giá đ..."
1,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",Gia Miêu,Bất động sản,"['Thủ Thiêm', 'Đấu giá đất']"
2,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,"['Dự án', 'Nhà ở xã hội', 'Dự án nhà ở xã hội'..."
3,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",Kim Nhung (T/H),Bất động sản,"['Giấy phép xây dựng', 'Hồ sơ hoàn công', 'nhà..."
4,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",Huyền Nguyễn,Bất động sản,['An Quý Villa']
...,...,...,...,...,...,...,...
313303,Quảng Trị: Trao 300 triệu đồng cho người dân b...,"Sáng nay (7.10) tại tỉnh Quảng Trị, đồng chí N...",Đoàn cứu trợ đã trực tiếp về huyện Vĩnh Linh v...,"Thứ hai, 07/10/2013 12:02 (GMT+7)",Hưng Thơ,Tin hoạt động,"['Quảng Trị', 'Bão']"
313304,Trao 100 triệu đồng hỗ trợ CNVCLĐ tỉnh Nghệ An...,"Sáng nay (7.10), tại Nghệ An, Quỹ Tấm lòng vàn...",Đồng chí Phạm Anh Tuấn cũng cho biết: Chiều qu...,"Thứ hai, 07/10/2013 11:53 (GMT+7)",Bảo Duy,Tin hoạt động,"['Nghệ An', 'Bão']"
313306,Quỹ Tấm lòng vàng Lao Động: Trao 4 triệu đồng ...,Đại diện Quỹ Tấm lòng Lao Động và LĐLĐ Phú Yên...,Số tiền trên được trích từ nguồn Quỹ Tấm lòng ...,"Chủ nhật, 06/10/2013 19:33 (GMT+7)",L.Phong,Tin hoạt động,"['Quỹ Tấm lòng Vàng', 'Quỹ Tấm lòng vàng Lao Đ..."
313307,Thêm những tấm lòng thơm thảo hướng về đồng bà...,"Tính đến chiều 5.10, Quỹ Tấm lòng vàng (TLV) L...","Theo đó, Quỹ TLV Lao Động quyết định hỗ trợ 6...","Thứ bảy, 05/10/2013 15:51 (GMT+7)",Bảo Duy,Tin hoạt động,"['Miền Trung', 'Đồng bào miền Trung']"


In [5]:
#Đếm số lượng Category
num_unique_categories = df['Category'].nunique()

print(f'Số lượng giá trị duy nhất trong cột Category: {num_unique_categories}')

# Lấy các giá trị duy nhất trong cột Category
unique_categories = df['Category'].unique()

# Hiển thị kết quả
print(f'Các giá trị duy nhất trong cột Category: {unique_categories}')

# Nhóm các hàng theo giá trị của cột Category và lấy 1300 dòng đầu tiên cho mỗi nhóm
result = df.groupby('Category').head(3000)

# Hiển thị kết quả
result

Số lượng giá trị duy nhất trong cột Category: 39
Các giá trị duy nhất trong cột Category: ['Bất động sản' 'Xã hội' 'Lao Động cuối tuần' 'Kinh doanh' 'Thời sự'
 'Công đoàn' 'Bạn đọc' 'Pháp luật' 'Sự kiện Bình luận' 'Xe +' 'Thế giới'
 'Văn hóa - Giải trí' 'Sổ tay kinh tế' 'Lao Động & Đời sống' 'Phóng sự'
 'Lưu trữ' 'Tin địa phương' 'Tin bài liên quan' 'Giáo dục' 'Sức khỏe'
 'Thể thao' 'Media' 'Video' 'Diễn đàn' 'Tấm Lòng Vàng' 'Công nghệ'
 'Thông tin doanh nghiệp' 'Tin bài nổi bật' 'Gia đình - Hôn nhân'
 'Người Việt tử tế' 'Thông tin tiện ích' 'Tin tức việc làm'
 'Lao Động Xuân' 'Tin hoạt động' 'Du lịch' 'Tin bài xem thêm'
 'Tản mạn - Chuyện dọc đường' 'Phóng sự - Điều tra' 'Quỹ TLV']


,Title,Summary,Contents,Date,Author(s),Category,Tags
0,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",Phương Nhiên,Bất động sản,"['Lâm Đồng', 'Ngọc Trinh', 'Chiêu trò', 'Giá đ..."
1,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",Gia Miêu,Bất động sản,"['Thủ Thiêm', 'Đấu giá đất']"
2,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,"['Dự án', 'Nhà ở xã hội', 'Dự án nhà ở xã hội'..."
3,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",Kim Nhung (T/H),Bất động sản,"['Giấy phép xây dựng', 'Hồ sơ hoàn công', 'nhà..."
4,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",Huyền Nguyễn,Bất động sản,['An Quý Villa']
...,...,...,...,...,...,...,...
313303,Quảng Trị: Trao 300 triệu đồng cho người dân b...,"Sáng nay (7.10) tại tỉnh Quảng Trị, đồng chí N...",Đoàn cứu trợ đã trực tiếp về huyện Vĩnh Linh v...,"Thứ hai, 07/10/2013 12:02 (GMT+7)",Hưng Thơ,Tin hoạt động,"['Quảng Trị', 'Bão']"
313304,Trao 100 triệu đồng hỗ trợ CNVCLĐ tỉnh Nghệ An...,"Sáng nay (7.10), tại Nghệ An, Quỹ Tấm lòng vàn...",Đồng chí Phạm Anh Tuấn cũng cho biết: Chiều qu...,"Thứ hai, 07/10/2013 11:53 (GMT+7)",Bảo Duy,Tin hoạt động,"['Nghệ An', 'Bão']"
313306,Quỹ Tấm lòng vàng Lao Động: Trao 4 triệu đồng ...,Đại diện Quỹ Tấm lòng Lao Động và LĐLĐ Phú Yên...,Số tiền trên được trích từ nguồn Quỹ Tấm lòng ...,"Chủ nhật, 06/10/2013 19:33 (GMT+7)",L.Phong,Tin hoạt động,"['Quỹ Tấm lòng Vàng', 'Quỹ Tấm lòng vàng Lao Đ..."
313307,Thêm những tấm lòng thơm thảo hướng về đồng bà...,"Tính đến chiều 5.10, Quỹ Tấm lòng vàng (TLV) L...","Theo đó, Quỹ TLV Lao Động quyết định hỗ trợ 6...","Thứ bảy, 05/10/2013 15:51 (GMT+7)",Bảo Duy,Tin hoạt động,"['Miền Trung', 'Đồng bào miền Trung']"


In [6]:
#Viết hoa cột Author(s)
df.loc[:, "Author(s)"] = df["Author(s)"].str.upper()


In [8]:
df = result
backup = df.copy()
df

,Title,Summary,Contents,Date,Author(s),Category,Tags
0,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",Phương Nhiên,Bất động sản,"['Lâm Đồng', 'Ngọc Trinh', 'Chiêu trò', 'Giá đ..."
1,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",Gia Miêu,Bất động sản,"['Thủ Thiêm', 'Đấu giá đất']"
2,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,"['Dự án', 'Nhà ở xã hội', 'Dự án nhà ở xã hội'..."
3,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",Kim Nhung (T/H),Bất động sản,"['Giấy phép xây dựng', 'Hồ sơ hoàn công', 'nhà..."
4,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",Huyền Nguyễn,Bất động sản,['An Quý Villa']
...,...,...,...,...,...,...,...
313303,Quảng Trị: Trao 300 triệu đồng cho người dân b...,"Sáng nay (7.10) tại tỉnh Quảng Trị, đồng chí N...",Đoàn cứu trợ đã trực tiếp về huyện Vĩnh Linh v...,"Thứ hai, 07/10/2013 12:02 (GMT+7)",Hưng Thơ,Tin hoạt động,"['Quảng Trị', 'Bão']"
313304,Trao 100 triệu đồng hỗ trợ CNVCLĐ tỉnh Nghệ An...,"Sáng nay (7.10), tại Nghệ An, Quỹ Tấm lòng vàn...",Đồng chí Phạm Anh Tuấn cũng cho biết: Chiều qu...,"Thứ hai, 07/10/2013 11:53 (GMT+7)",Bảo Duy,Tin hoạt động,"['Nghệ An', 'Bão']"
313306,Quỹ Tấm lòng vàng Lao Động: Trao 4 triệu đồng ...,Đại diện Quỹ Tấm lòng Lao Động và LĐLĐ Phú Yên...,Số tiền trên được trích từ nguồn Quỹ Tấm lòng ...,"Chủ nhật, 06/10/2013 19:33 (GMT+7)",L.Phong,Tin hoạt động,"['Quỹ Tấm lòng Vàng', 'Quỹ Tấm lòng vàng Lao Đ..."
313307,Thêm những tấm lòng thơm thảo hướng về đồng bà...,"Tính đến chiều 5.10, Quỹ Tấm lòng vàng (TLV) L...","Theo đó, Quỹ TLV Lao Động quyết định hỗ trợ 6...","Thứ bảy, 05/10/2013 15:51 (GMT+7)",Bảo Duy,Tin hoạt động,"['Miền Trung', 'Đồng bào miền Trung']"


In [9]:
# Tạo một danh sách stopword đơn giản cho tiếng Việt
stop_words = set(["là", "của", "và", "có", "được", "trong", "với", "cho", "đến"])
# Lấy một danh sách stopword cho tiếng Việt từ file
with open("vietnamese-stopwords.txt", "r", encoding="utf-8") as f:
    new_stop_words = set(f.read().splitlines())
    stop_words.update(new_stop_words)

# Định nghĩa hàm chuẩn hóa dữ liệu văn bản
def normalize_text(text):
    # Loại bỏ ký tự đặc biệt và chuyển đổi sang chữ thường
    text = "".join(char for char in text if char.isalnum() or char.isspace()).lower()
    # Tách từ
    words = word_tokenize(text)
    # Loại bỏ stopword
    words = [word for word in words if word not in stop_words]
    # Nối lại các từ đã xử lý
    return " ".join(words)

# Chuẩn hóa dữ liệu văn bản cho cột "Title" và "Summary"
df.loc[:, "Title"] = df["Title"].apply(normalize_text)
df.loc[:, "Summary"] = df["Summary"].apply(normalize_text)


In [16]:
df

,index_0,index_1,index_2,index_3,index_4,index_5,index_6,index_7,index_8,index_9
0,0,2338,870,926,414,2467,2883,2025,2591,2264
1,1,169,15,474,573,79,562,11,683,618
2,2,232,22696,22678,2779,2398,22,1223,2351,536
3,3,341,2545,2370,1266,2791,1540,462,1137,2852
4,4,1121,807,808,1494,1035,9,896,1013,1427
...,...,...,...,...,...,...,...,...,...,...
51139,51139,51140,50352,51108,50290,51052,51121,51135,51129,51069
51140,51140,50770,51142,51110,51112,50898,51128,51127,51132,50808
51141,51141,50936,51110,51066,51069,17781,50114,51134,51107,50945
51142,51142,50770,51140,51128,51127,50898,51132,51112,50808,51110


In [10]:
# Khởi tạo vectorizer
vectorizer = TfidfVectorizer()

# Tính toán TF-IDF cho tất cả các thuộc tính
vectorizer.fit(df['Title'] + df['Summary'])
title_tfidf = vectorizer.transform(df['Title'])
summary_tfidf = vectorizer.transform(df['Summary'])

# Kết hợp các ma trận TF-IDF lại với nhau
# (có thể sử dụng các phương pháp khác nhau để kết hợp)
features = hstack([title_tfidf, summary_tfidf])

In [11]:
# Khởi tạo encoder
encoder = OneHotEncoder()

# Mã hóa các thuộc tính phân loại
category_encoded = encoder.fit_transform(df[['Category']])
author_encoded = encoder.fit_transform(df[['Author(s)']])

# Kết hợp các ma trận lại với nhau
features = hstack([features, category_encoded])
features = hstack([features, author_encoded])

In [17]:
features

<51144x119540 sparse matrix of type '<class 'numpy.float64'>'
	with 1835496 stored elements in Compressed Sparse Row format>

In [12]:
#Features là ma trận đặc trưng đã tính toán ở bước trước

# Khởi tạo mô hình
model = NearestNeighbors()

# Huấn luyện mô hình
model.fit(features)

# Dự đoán mức độ tương tự giữa các bài viết
# (ví dụ: tìm 10 bài viết tương tự nhất với bài viết đầu tiên)
distances, indices = model.kneighbors(features, n_neighbors=10)

In [28]:
model

NearestNeighbors()

In [13]:
print("Indices:", indices)
print("Distances:", distances)

Indices: [[    0  2338   870 ...  2025  2591  2264]
 [    1   169    15 ...    11   683   618]
 [    2   232 22696 ...  1223  2351   536]
 ...
 [51141 50936 51110 ... 51134 51107 50945]
 [51142 50770 51140 ... 51112 50808 51110]
 [51143 51130 51138 ... 51135 51126 51142]]
Distances: [[0.         2.15494129 2.19149429 ... 2.20461091 2.20910576 2.213327  ]
 [0.         1.49470727 1.51455631 ... 1.63436612 1.63960173 1.64644031]
 [0.         1.93750148 2.18677397 ... 2.24914565 2.25131046 2.25488602]
 ...
 [0.         2.2299281  2.27719619 ... 2.30987099 2.31197978 2.31481786]
 [0.         1.80704948 1.85619937 ... 1.95240906 1.95925062 1.96043152]
 [0.         1.40640087 1.69577633 ... 2.13220108 2.16481757 2.17629431]]


In [14]:
split_indices = np.split(indices, indices.shape[1], axis=1)

# Tạo DataFrame từ các mảng con
data = {f'index_{i}': split_indices[i].ravel() for i in range(len(split_indices))}
df = pd.DataFrame(data)

# Xuất DataFrame ra tệp CSV
df.to_csv('ket_qua.csv', index=False, header=False)

In [15]:
df
df.to_csv('ket_qua.csv', index=False, header=False)

In [26]:
backup.iloc[3333]

Title        Sắp có khách sạn mang thương hiệu Travelodge đ...
Summary      Ông Jonathan Wigle - Giám đốc điều hành Travel...
Contents     Dự án The Arena Cam Ranh sẽ mang thương hiệu T...
Date                         Thứ hai, 05/03/2018 10:00 (GMT+7)
Author(s)                                                 T.H 
Category                                            Kinh doanh
Tags         ['Dự án The Arena Cam Ranh khai trương năm 2020']
Name: 6064, dtype: object

In [27]:
backup.iloc[3301]

Title        Long An: Hàng trăm khách hàng sập bẫy đất nền ...
Summary      Trong đơn khiếu nại gửi tới báo Lao Động, hơn ...
Contents     Cụ thể theo phản ánh của các khách hàng mua dự...
Date                          Thứ tư, 23/05/2018 15:24 (GMT+7)
Author(s)                                           Bảo Chương
Category                                            Kinh doanh
Tags         ['Dự án Everluck Residence', 'Dự án bị kê biên...
Name: 5990, dtype: object

In [140]:
import ast

def convert_to_list(tags_str):
    try:
        tags_list = ast.literal_eval(tags_str)
        if isinstance(tags_list, list):
            return tags_list
        else:
            return None
    except:
        return None

df['Tags'] = df['Tags'].apply(convert_to_list)

C:\Users\votha\AppData\Local\Temp\ipykernel_21384\3373366110.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tags'] = df['Tags'].apply(convert_to_list)


In [141]:
def format_tags(tags):
    tags_str = 'tags:\n'
    for tag in tags:
        tags_str += f'    - {tag}\n'
    return tags_str

df['Tags'] = df['Tags'].apply(format_tags)


C:\Users\votha\AppData\Local\Temp\ipykernel_21384\2745699286.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tags'] = df['Tags'].apply(format_tags)


In [142]:
df

,URL,Title,Summary,Contents,Date,Author(s),Category,Tags
0,https://laodong.vn/bat-dong-san/thong-tin-ngoc...,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",Phương Nhiên,Bất động sản,tags:\n - Lâm Đồng\n - Ngọc Trinh\n -...
1,https://laodong.vn/bat-dong-san/lo-hong-trong-...,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",Gia Miêu,Bất động sản,tags:\n - Thủ Thiêm\n - Đấu giá đất\n
2,https://laodong.vn/bat-dong-san/som-hoan-thien...,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,tags:\n - Dự án\n - Nhà ở xã hội\n - ...
3,https://laodong.vn/bat-dong-san/chi-tiet-ho-so...,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",Kim Nhung (T/H),Bất động sản,tags:\n - Giấy phép xây dựng\n - Hồ sơ h...
4,https://laodong.vn/bat-dong-san/khoi-tao-khong...,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",Huyền Nguyễn,Bất động sản,tags:\n - An Quý Villa\n
...,...,...,...,...,...,...,...,...
995,https://laodong.vn/bat-dong-san/can-cu-giai-qu...,Căn cứ giải quyết tranh chấp về lối đi khi khô...,Dưới đây là căn cứ giải quyết tranh chấp về lố...,Căn cứ giải quyết tranh chấp về lối đi khi khô...,"Thứ bảy, 27/11/2021 08:30 (GMT+7)",Trang Thiều (T/H),Bất động sản,tags:\n - Giấy chứng nhận quyền sử dụng đất...
996,https://laodong.vn/bat-dong-san/thanh-hoa-don-...,Thanh Hóa đón dòng sản phẩm mới Shop Luxury Eu...,Eurowindow Holding - nhà phát triển dự án Euro...,Shop thương mại lâu nay vẫn là “miếng bánh ngo...,"Thứ sáu, 26/11/2021 19:00 (GMT+7)",H.Ly,Bất động sản,tags:\n - Eurowindow Holding\n - Eurowin...
997,https://laodong.vn/bat-dong-san/bat-dong-san-y...,Bất động sản Yên Bái nhộn nhịp đón nhà đầu tư,Nhờ chiến lược tập trung phát triển đô thị bài...,"Những năm qua, Yên Bái đã là một trong những t...","Thứ sáu, 26/11/2021 19:00 (GMT+7)",H.Ly,Bất động sản,tags:\n - Eurowindow Holding\n - Eurowin...
998,https://laodong.vn/bat-dong-san/tphcm-khong-ta...,TPHCM không tăng giá đất năm 2022 vì dịch COVI...,TPHCM thống nhất giữ nguyên hệ số điều chỉnh g...,Văn phòng UBND TPHCM vừa thông báo kết luận củ...,"Thứ sáu, 26/11/2021 16:37 (GMT+7)",MINH QUÂN,Bất động sản,tags:\n - Bảng giá đất TPHCM\n - Dịch CO...


In [143]:
df['Contents'] = df['Contents'].str.replace('  ', '\n\n', regex=False)

C:\Users\votha\AppData\Local\Temp\ipykernel_21384\1218975678.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Contents'] = df['Contents'].str.replace('  ', '\n\n', regex=False)


In [144]:
import re

df['title'] = df['Title'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

C:\Users\votha\AppData\Local\Temp\ipykernel_21384\3395952097.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['Title'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


In [145]:
!pip install unidecode
import unidecode

df['title'] = df['title'].apply(lambda x: unidecode.unidecode(x).replace(' ', '-'))

C:\Users\votha\AppData\Local\Temp\ipykernel_21384\69963698.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].apply(lambda x: unidecode.unidecode(x).replace(' ', '-'))

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\votha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [146]:
df

,URL,Title,Summary,Contents,Date,Author(s),Category,Tags,title
0,https://laodong.vn/bat-dong-san/thong-tin-ngoc...,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",Phương Nhiên,Bất động sản,tags:\n - Lâm Đồng\n - Ngọc Trinh\n -...,Thong-tin-Ngoc-Trinh-mua-dat-o-Bao-Loc-chi-la-...
1,https://laodong.vn/bat-dong-san/lo-hong-trong-...,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",Gia Miêu,Bất động sản,tags:\n - Thủ Thiêm\n - Đấu giá đất\n,Lo-hong-trong-viec-tham-tra-nang-luc-tai-chinh...
2,https://laodong.vn/bat-dong-san/som-hoan-thien...,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,tags:\n - Dự án\n - Nhà ở xã hội\n - ...,Som-hoan-thien-cac-du-an-nha-o-xa-hoi-de-CNLD-...
3,https://laodong.vn/bat-dong-san/chi-tiet-ho-so...,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",Kim Nhung (T/H),Bất động sản,tags:\n - Giấy phép xây dựng\n - Hồ sơ h...,Chi-tiet-ho-so-hoan-cong-nha-o-nam-2022
4,https://laodong.vn/bat-dong-san/khoi-tao-khong...,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",Huyền Nguyễn,Bất động sản,tags:\n - An Quý Villa\n,Khoi-tao-khong-gian-song-dang-cap-don-song-dau...
...,...,...,...,...,...,...,...,...,...
995,https://laodong.vn/bat-dong-san/can-cu-giai-qu...,Căn cứ giải quyết tranh chấp về lối đi khi khô...,Dưới đây là căn cứ giải quyết tranh chấp về lố...,Căn cứ giải quyết tranh chấp về lối đi khi khô...,"Thứ bảy, 27/11/2021 08:30 (GMT+7)",Trang Thiều (T/H),Bất động sản,tags:\n - Giấy chứng nhận quyền sử dụng đất...,Can-cu-giai-quyet-tranh-chap-ve-loi-di-khi-kho...
996,https://laodong.vn/bat-dong-san/thanh-hoa-don-...,Thanh Hóa đón dòng sản phẩm mới Shop Luxury Eu...,Eurowindow Holding - nhà phát triển dự án Euro...,Shop thương mại lâu nay vẫn là “miếng bánh ngo...,"Thứ sáu, 26/11/2021 19:00 (GMT+7)",H.Ly,Bất động sản,tags:\n - Eurowindow Holding\n - Eurowin...,Thanh-Hoa-don-dong-san-pham-moi-Shop-Luxury-Eu...
997,https://laodong.vn/bat-dong-san/bat-dong-san-y...,Bất động sản Yên Bái nhộn nhịp đón nhà đầu tư,Nhờ chiến lược tập trung phát triển đô thị bài...,"Những năm qua, Yên Bái đã là một trong những t...","Thứ sáu, 26/11/2021 19:00 (GMT+7)",H.Ly,Bất động sản,tags:\n - Eurowindow Holding\n - Eurowin...,Bat-dong-san-Yen-Bai-nhon-nhip-don-nha-dau-tu
998,https://laodong.vn/bat-dong-san/tphcm-khong-ta...,TPHCM không tăng giá đất năm 2022 vì dịch COVI...,TPHCM thống nhất giữ nguyên hệ số điều chỉnh g...,Văn phòng UBND TPHCM vừa thông báo kết luận củ...,"Thứ sáu, 26/11/2021 16:37 (GMT+7)",MINH QUÂN,Bất động sản,tags:\n - Bảng giá đất TPHCM\n - Dịch CO...,TPHCM-khong-tang-gia-dat-nam-2022-vi-dich-COVID19


In [147]:
re = pd.read_csv("Output.csv")
re


,0,870,714,926,630
0,1,169,15,474,11
1,2,232,457,613,720
2,3,341,132,796,175
3,4,808,807,9,84
4,5,979,174,722,951
...,...,...,...,...,...
994,995,971,992,760,747
995,996,852,939,943,874
996,997,981,894,852,939
997,998,654,872,430,460


In [148]:
value = re.iloc[0, 1]
print(value)

169


In [152]:
for i, row in df.iterrows():
    with open(f'{row["title"]}.md', 'w', encoding='utf-8') as md_file:
        md_file.write('---\n')
        md_file.write(f'id: \'{i}\'\n')
        md_file.write(f'related_1: \'{re.iloc[i, 1]}\'\n')
        md_file.write(f'related_2: \'{re.iloc[i, 2]}\'\n')
        md_file.write(f'related_3: \'{re.iloc[i, 3]}\'\n')
        md_file.write(f'related_4: \'{re.iloc[i, 4]}\'\n')
        md_file.write(f'postFormat: \'standard\'\n')
        md_file.write(f'title: \'{row["Title"]}\'\n')
        md_file.write(f'excerpt: \'{row["Summary"]}\'\n')
        md_file.write(f'featureImg: \'/images/posts/post_1.jpg\'\n')
        md_file.write(f'date: \'{row["Date"]}\'\n')
        md_file.write(f'cate: \'{row["Category"]}\'\n')
        md_file.write(f'cate_bg: \'bg-color-blue-one\'\n')
        md_file.write(f'cate_img: \'/images/category/technology.png\'\n')
        md_file.write(f'post_views: \'0 Views\'\n')
        md_file.write(f'post_share: \'0 Shares\'\n')
        md_file.write(f'author_name: \'{row["Author(s)"]}\'\n')
        md_file.write(f'author_desg: \'Publisher\'\n')
        md_file.write(f'author_img: \'/images/author/ashley_graham.png\'\n')
        md_file.write(f'author_bio: \'At 29 years old, my favorite compliment is being told that I look like my mom. Seeing myself in her image, like this daughter up top, makes me so proud of how far I’ve come, and so thankful for where I come from.\'\n')
        md_file.write(f'author_social:\n    -\n        icon: fab fa-facebook-f\n        url: https://facebook.com/\n    -\n        icon: fab fa-twitter\n        url: https://twitter.com/\n')
        md_file.write(f'{row["Tags"]}\n')
        md_file.write(f'---\n\n')
        md_file.write(f'{row["Contents"]}')

IndexError: index 999 is out of bounds for axis 0 with size 999

In [ ]:
##END